# Graphical Models Part 5: the Junction Tree Algorithm

This document is about the Junction Tree Algorithm, and its simpler forms (sum-product)

## Abbreviations
GraphNote = [Undirected Graphical Models: Chordal Graphs, Decomposable Graphs, Junction Trees, and Factorizations](graphnotes.pdf) Original link at <https://people.eecs.berkeley.edu/~bartlett/courses/2009fall-cs281a/graphnotes.pdf>

---

## Cluster Graph, Cluster Tree, Clique Tree, Junction Tree

Notice that here we always assume the underlying graph is connected. As otherwise we can tackle them separately.

Fxxk Koller, who introduces so many concepts!

First there are some different definitions for certain terms. Here I will unify them.

* **cluster tree** (Koller) and **clique tree** (Jordan or GraphNote) are the same. I always stick to clique tree.
* **clique tree** (Koller) and **junction tree** (Jordan or GraphNote) are the same. I always stick to junction tree.

So actually there are only three different structures, in terms of generality.

All these definitions are defining some undirected graph $G_T=(V_T,E_T)$ w.r.t over some undirected graph $G=(V,E)$, where each node in this new graph is some clique in $G$, and all maximal cliques are nodes in $G_T$ (family preservation). Notice that here, we don't define the edges $E_T$ (sepset) yet. The below definitions are consistent with those given in GraphNote and Jordan's book, and may deviate from Koller's in some case (such as whether we require sepset to be intersection, for definition of *clique tree* (cluster tree in Koller)).

* Most general, we have *cluster graph* (Def 10.1 of Koller's book), which makes least requirement on form of $E_T$, only requring that if an edge exists, then variables on edge (sepset) exists on both nodes it links.
* Then we have *clique tree* (cluster tree in Koller), where if an edge exists, then its variables (sepset) must be the intersection of variables from two nodes.
* Finally we have *junction tree* (clique tree in Koller), where running intersection property must be satisfied (or junction tree property in Jordan's book).

Notice that given two nodes in $V_T$ with shared variables, it's not necessary that they are directly linked. All the three above structures can arbitrarily define whether we have an edge between two nodes or not, as long as the final structure is a tree (for the case of clique tree and junction tree).


### bottom line

If you don't want to get confused, stick to definitions given in GraphNote first.

---

### Cluster Graph, Clique Tree
Of all these, cluster graph is the most general one.

There are two versions of definition.

First, the one in Koller's book (pp. 346), Def. 10.1:
![image](./JT/defs/cluster_graph_Koller.png)
But in Koller's Coursera course, she added that Running Intersection Property should be maintained as well, that is 
![image](./JT/defs/RIP_Koller_Coursera.png) Here, this RIP is defined on cluster graph, not on cluster tree. So she says "exists a unique path", which is redundant in the case of tree.

A version of RIP for clique tree (cluster tree in Koller) is given in pp. 347 of Koller's book, Def. 10.2:
![image](./JT/defs/RIP_Koller.png)
This definition of RIP is confusing, I think it should specify that $X$ should exist on all the sepsets on the unique path, as well, to match the definition given in Coursera.

Another equivalent definition of RIP is that given any variable $X$, sepsets and cliques containing it form a tree (Koller Coursera).
![image](./JT/defs/RIP_Koller_Coursera_2.png)

So I'm not sure whether there's RIP or not in the definition of cluster graph. But I believe in practice, we need it for algorithm to work.


Clique tree (cluster tree in Koller) is just a tree-structured cluster graph. In the definitions given by Jordan and GraphNote, RIP is not assumed, but in addition, sepset must be intersection of variables.

### Junction Tree
Junction Tree is Clique Tree in Koller's book. It's a tree-structured cluster graph with family preservation and running intersection property.

One definition is given in Def 10.3 (pp. 348):
![image](./JT/defs/clique_tree_Koller_1.png)

Another "definition" of junction tree is given in pp. 140 of Koller's book.
![image](./JT/defs/clique_tree_Koller_2.png)

IMO, this definition using separation is ill-defined, as discussed in my note for chapter 4 of Koller's book.

Koller claimed that these two definitions are actually equivalent (pp. 348):
![image](./JT/thms/clique_tree_2_def_equi_comment_Koller.png)
![image](./JT/thms/clique_tree_2_def_equi_Koller.png)

I would rather say, remove the second definition, and only keep the Theorem 10.2 in one direction, that is, if a clique tree has RIP (which becomes a junction tree), then its sepset will seprate variables on its two sides apart.

This property of junction tree about separation gives insight on where the complexity of JT algorithm is hidden: size of sepset. The minimal size of the max sepset can be pretty big for some graphs. This is discussed in her Coursera course (pp. 56 of `Section-3-Belief-Propagation.pdf`).
![image](./JT/thms/clique_tree_Koller_2_implication.png)


### Proof

The proof is also given in her book's solution manual.

![image](./JT/proofs/thm10_2_one_direction_ref.png)

Notice that, to make this proof work, it's best to first "normalize" the tree, using Theorem 10.6 of Koller's book. So that no matter what sepset we choose, it always separates two non empty sets of variables apart.

![image](./JT/thms/clique_tree_reduction_Koller.png)


### (GARBAGE) my previous attempt at for the other direction of "theorem 10.2"

> After proving this, we now have to show that there exists a chordal graph $H$ consisting the original moral graph $H_\phi$ and its maximal cliques are all represented by the nodes of the clique tree, and each node is a clique in $H$. Also, the sepset independence statement should still hold in $H$, to match the definition of clique tree in pp. 140. Here Koller doesn't specify explicitly what that $H$ is.
>
> But I think $H$ is simply the graph $H'$ induced from the given clique tree. This is created by connecting all nodes in clusters. Or, we connect an edge between two nodes if and only if they occur in some cluster in the clique tree.
>
> Also trivially this $H'$ consists of $H_\phi$, since the former has possibly more edges than the latter. In addition, I need to show two things.
>
> 1. This graph is chordal. This can be shown by showing it being decomposable, since being chordal and being decomposable are equivalent ([graph notes](./graphnotes.pdf)). Why clique tree is decomposable? First, we can get an equivalent clique tree that contains no cluster that is a subset of another (pp. 373 of Koller's book).
> ![image](./JT/thms/clique_tree_reduction_Koller.png)
This means that every sepset divides the variables into 3 disjoint, non-empty parts. Then, we can assume that the original distribution that this clique tree works on factorizes by the cliques in the clique tree. So $H_\phi = H'$ and clearly Thm 10.2 works for $H'$. This means that (a)-(d) for being decomposable on pp. 1 of [graph notes](./graphnotes.pdf) are satisfied. Then I have to show that that the subgraph for the union of sepset $S$ plus any of two other groups of variables  $A,B$ on $H'$ is still decomposable. This is shown by (1) each clique in the clique tree is decomposable; (2) the subgraphs are exactly the induced graphs for two sub clique trees separated by the sepset. First, dividing a clique tree into two parts based on one sepset indeed gives you two clique trees (proof omitted) Then consider the subgraph for union of sepset and group of variable on one (say, left) side of sepset. Every edge between two nodes must originally come from a factor that connect them, and this factor must be able to put in that (say, left) side of the clique tree. So the induced graph of sub clique trees contain the subgraph of $H'$ for $S,A$ (or $S,B$). In the other direction, every edge in the induced subgraph must exist in the subgraph of $H'$ for $S,A$ (or $S,B$) as well. This is because every edge in the induced subgraph must exist in the original induced graph $H'$, and it connects variables among $S,A$ (or $S,B$). Therefore, when taking the subgraph for $S,A$ (or $S,B$), this edge must be preserved as well. So induced subgraph is the same as subgraph of induced graph. Since the base case (induced graph for one clique) is decomposable, all clique trees are decomposable.
> 2. I need to show that cliques in the tree are cliques in $H'$ (trivial by definition) and that max cliques in $H'$ are cliques in the tree. The latter statement holds because if there are any max clique in $H'$ not in in the tree, then it must involves variables $a,b$ that no clique in the clique tree have them all. But this implies, by the way we construct $H'$, $a,b$ should be in some clique in the clique tree. So by contradiction, we show the latter statement.

---

## Junction Tree Algorithm

There are two equivalent algorithms, Shafer-Shenoy and Hugin.

### Shafer-Shenoy algorithm

I will first present the version in Koller's book. This is not the divisive version in Jordan's book. Here, only messages between cliques are changing, but clique potentials themselves are not. In the end, we multiply the clique potential with incoming messages to get the final belief. This is also called **Shafer-Shenoy** algorithm, in constrast to the divisive one, which is called **Hugin** or **Lauritzen-Spiegelhalter** algorihtm (which is called **Belief Update** in Koller's book).

The algorithm is simple: pick a root node, do a upward pass of message passing to the root, and do downward pass of message passing to all non-roots.

The upward pass is given in Algorithm 10.1 of Koller's book, and the upward-downward combined version is represented in an asynchronous algorithm (10.2).

Algo 10.1 (pp. 353, Koller's book):
![image](./JT/algos/JT_Algo10_1_Koller.png)

Algo 10.2 and the argument that this algorithm is equivalent to a 2-pass version (pp. 357, Koller's book):
![image](./JT/algos/JT_Algo10_2_Koller.png)

#### Important properties.

The take home message is that each local belief is now the correct (unnormalized) marginal probability (pp. 357, Koller's book).
![image](./JT/thms/JT_property_marginal_Koller.png)
It's unnormalized in the sense that if the product of all factors has a non-unit partition function Z, then this marginal probability also has to be divided with the same partition function Z to make it a normalized marginal probability. This assumes that during message passing, you don't do any normalization by yourself. If you do that, the local beliefs are still marginal probabilities, but maybe they have some other partition functions. Basically, no matter you do normalization on the fly or not, the correctness of the algorithm won't be affected. Since it's essentially doing variable elimination, plus some normalization factors, which won't affect the relative magnitudes of different configurations of variables.

Also, after running the algorithm 10.2, the clique tree is calibrated (pp. 358, Koller's book)
![image](./JT/defs/calibrated_def_Koller.png)
and each sepset term has three equivalent representations: marginalization from two connected clique terms, or the product of messages. (pp. 361, Koller's book)
![image](./JT/proofs/calibrated_sepset_Koller.png)

This means that we can rewrite any distribution defined on a clique tree, using calibrated potentials and sepset terms, and they all correspond to (unnormalized) marginals (pp. 364, Koller's book).
![image](./JT/thms/JT_marginal_representation_Koller.png)


If we just define sepset terms $\mu$ to be product of messages. Then during any step of the algorithm 10.2, we should always have that the product of clique beliefs divided by product of sepset terms being equal to the product of original factors. This is called **clique tree invariant**. (pp. 361-362, Koller's book)
![image](./JT/clique_tree_reparameterization_1_Koller.png)
![image](./JT/clique_tree_reparameterization_2_Koller.png)

It's also discussed in her Coursera course. It's a reparameterization of original factors. This means message passing involves no information loss (Koller's Coursera course, pp. 28 of `Section-3-Belief-Propagation.pdf`, green note on top) 
![image](./JT/proofs/clique_tree_reparameterization_Koller_Coursera.png)


In her book, actually there's no definition of **convergence**, which is given in her Coursera course (pp. 26 of  `Section-3-Belief-Propagation.pdf`)
![image](./JT/defs/clique_tree_convergence_Koller_Coursera.png)

Well I personally think it's useless to differentiate **calibrated** and **convergence** from a theoretic point of view. Simply say **when the algorithm stops** (after two passes).


### Hugin
I will provide some introduction on Hugin algorithm, since it's essentially the same as the Shafer-Shenoy algorithm. Hugin is simply the two rules plus a protocol on order of passing to preserve consistency. For proof that it's equivalent to Shafer-Shenoy, check Koller's book (Thm 10.5, pp. 368). Basically, after each update, we can find isomorphism between two algorithms.

Two rules (pp. 16 of Chapter 17, Jordan's book):
![image](./JT/algos/JT_Hugin_update_Jordan.png)

The message passing protocol (pp. 18 of Chapter 17, Jordan's book):
![image](./JT/algos/JT_Hugin_protocol_Jordan.png)

One algorithm implementing this protocol (pp. 19-20, Jordan's book, Chapter 17):
![image](./JT/algos/JT_Hugin_protocol_implementation_1_Jordan.png)
![image](./JT/algos/JT_Hugin_protocol_implementation_2_Jordan.png)

---

### Property about calibrated clique tree

Finally, Koller's book says that for a calibrated tree, each clique belief proportional to marginal probability is equivalent to that the total **clique tree measure** proportional to the joint probability. You may wonder why we define this, since this **clique tree measure** is invariant when running a junction tree algorithm. I think here it doesn't matter how you get it (possibly through junction tree algorithm; or you can assign the numbers for each clique and sepset arbitrarily, and somehow make it locally calibrated).

definition of clique tree measure (pp. 363, Koller's book)
![image](./JT/defs/clique_tree_measure_def_Koller.png)

the theorem and sketch of proof (pp. 363-364, Koller's book)
![image](./JT/clique_tree_measure_thm_1_Koller.png)
![image](./JT/clique_tree_measure_thm_2_Koller.png)

This is my interpretation of the theorem. Given a calibrated tree, no matter how you get it (possibly through junction tree algorithm; or you can assign the numbers for each clique and sepset arbitrarily, and somehow make it locally calibrated), then if the total clique tree measure is proportional to to the joint distribution, then each clique is proportional to the marginal.

#### MUST SEE

A very powerful corollary can be derived at the end of my proof below for this theorem: **You can pick a subtree of the clique tree, and then compute the clique tree measure w.r.t. to that subtree, and then result should be the marginal probability over variables covered by the sub clique tree.**

#### Proof

The proof in the book has some problem. Most importantly, the equation below "using the standard chain-rule argument" and the equation below that equation have typo. All $C_i$ should be $C_i - S_{i,p_r(i)}.$ Here I think $p_r(i) $ means the parent of $i$ when $r$ is the root.

Let me prove why is modified equation is correct. I don't discriminate $=$ and $\propto$, and for convenience I just write $=$, as that scaling factor shouldn't matter.

* First, I need the clique tree to be compact, that is, using Theorem 10.6.
* After selecting a root clique, then we can topologically sort all cliques. So that each clique's parent is before its children. Let one such sorting to be $C_0, C_1, \ldots, C_{n-1}$. Notice that $C_0$ must be the root clique, and all others have exactly one parent, and I denote the sepset between $C_i$ and its parent to be $S_i$.
* I would argue that $\{C_i-S_i\} \cup \{C_0\}$ partitioned all variables in the distribution (by partition, I mean no double count or missing).
    - no double counting. If $\{C_i-S_i\}$ and $\{C_j-S_j\}$ has some intersection $X$. That means $C_i$ and $C_j$ both have $X$, and by RIP, $X$ must also exist on the path between $C_i$ and $C_j$ in the tree. Clearly, there's unique path from $C_i$ to $C_j$. I would say this path must either include $C_i$'s parent or $C_j$'s parent. If neither, then we would conclude that $C_i$ is descendant of $C_j$ AND $C_j$ is descendant for $C_i$, which will cover one parent, and it's contradictory. So suppose this path has $C_i$'s parent. Then definitely $C_i$'s parent has $X$ by RIP, and this means $S_i$ has $X$, meaning our assumption is wrong. If some $\{C_i-S_i\}$ has intersection with root $C_0$, similar arguments applies, except that that we must know that the unique path has $C_i$'s parent, and proof is easier.
    - cover every variable. Notice that $\{C_i-S_i\} = \{C_i-pr(C_i)\}$, ($pr(C_i)$ is parent of $C_i$, whose index must be smaller than $i$) by our definition of sepset. Then consider $\{C_0\} \cup [\cup_{i=1}^k \{C_i\}]$, and $\{C_0\} \cup [\cup_{i=1}^k \{C_i - pr(C_i)\}]$. I would argue by induction on $k$ from $1$ to $n-1$ that these two equations are equal. when $k=1$, it's trivial, since $pr(C_i)$ must be $C_0$. When $k=j+1$, we know by induction $[\{C_0\} \cup [\cup_{i=1}^j \{C_i - pr(C_i)\}] = [\{C_0\} \cup [\cup_{i=1}^j \{C_i\}]$. Thus $[\{C_0\} \cup [\cup_{i=1}^{j+1} \{C_i - pr(C_i)\}] = [\{C_0\} \cup [\cup_{i=1}^j \{C_i\}] \cup \{C_{j+1} - pr(C_{j+1})\}$, and since index of $pr(C_{j+1})$ must be among $0$ to $j$, equality still holds. Taking $k=n-1$, we know that $\{C_i-S_i\} \cup \{C_0\}$ contains variables covered in all $C_i$, covering every variable. Notice that our proof works for intermediate values, such as $j=2$ as well. This lays the foundation why I can do partial summation later.
* Thus, you could write some block chain rule for all variables in $P$, as $P(X) = P(C_0)P(C_1 - S_1 \mid C_0)P(C_2-S_2\mid C_0, C_1 - S_1)$, etc. 
* By my previous argument about the equality of two different union of sets, we know that in the above chain rule, for a given term $C_k-S_k \mid XXX$, $XXX$, which is actually $[\cup_{i=0}^{k-1} \{C_i\}]$ contain all variables in $C_k$'s parent. In particular, its sepset $S_k$. Also, all $[\cup_{i=0}^{k-1} \{C_i\}]-S_k$ are on the parent side of the tree, $S_k$ separates $C_k-S_k$ and $[\cup_{i=0}^{k-1} \{C_i\}]-S_k$ (otherwise, by RIP, one of $[\cup_{i=0}^{k-1} \{C_i\}]-S_k$ should be in $C_k$, as well as $S_k$, and this is contradictory to that $[\cup_{i=0}^{k-1} \{C_i\}]-S_k$ has empty intersection with $S_k$.
* Thus, we can apply Theorem 10.2 to have $P(C_k-S_k \mid XXX) = P(C_k-S_k \mid S_k)$, and we have the nice chain rule representation of the distribution as given in the proof on book.
* For the if direction, it's trivial, since every clique potential is proprotional to marginal, and dividing it by the corresponding sepset gives the conditional.
* For the only if direction, notice that now the form of distribution has become
$$
P(X) = P(C_0) \prod_{i=1}^{n-1} P(C_i-S_i\mid S_i) = \beta_0(C_0) \prod_{i=1}^{n-1} \frac{\beta_i(C_i)}{\mu_{i, pr(i)}(S_i)}
$$
Notice that we are not sure (now) if $\frac{\beta_i(C_i)}{\mu_{i, pr(i)}(S_i)} = P(C_i-S_i\mid S_i)$, nor if $\beta_0(C_0) = P(C_0)$, but overall the product of all of them is same as (proportional to) $P(X)$.

Consider that we want to compute the marginal $P(C_0)$. we can get 
$$
\begin{align}
P(C_0) &= \beta_0(C_0) \sum_{X-C_0} \prod_{i=1}^{n-1} \frac{\beta_i(C_i)}{\mu_{i, pr(i)}(S_i)}\\
       &= \beta_0(C_0) \prod_{i=1}^{n-1} [\sum_{C_i-S_i}  \frac{\beta_i(C_i)}{\mu_{i, pr(i)}(S_i)}] \\
       & = \beta_0(C_0) \prod_{i=1}^{n-1} 1 = \beta_0(C_0)
\end{align}
$$
We first push summing inside product, and we know each must sum to the denominator, since $\mu_{i, pr(i)}(S_i)$ is calibrated.

We can repeat this procedure by choosing different root clique as $C_0$, and we can prove the result.

There might be some problems regarding the case of dividing by zero. I think 1) this is rarely encountered in practice; 2) I think if you define dividing by zero to be zero, as done in Koller's book or Jordan's book when discussion factor division, the theorems should still hold.

Notice that for the "only if" direction, during summation, if we don't marginalize $X-C_0$, but some smaller set, such as $X- \cup_{i=0}^j C_j$, then we can get
$$
\begin{align}
P(\cup_{i=0}^j C_j) &= \beta_0(C_0) \sum_{X-\cup_{i=0}^j C_j} \prod_{i=1}^{n-1} \frac{\beta_i(C_i)}{\mu_{i, pr(i)}(S_i)}\\
       &= \beta_0(C_0) \prod_{i=1}^{j} [\frac{\beta_i(C_i)}{\mu_{i, pr(i)}(S_i)}] \prod_{i=j+1}^{n-1} [\sum_{C_i-S_i}  \frac{\beta_i(C_i)}{\mu_{i, pr(i)}(S_i)}] \\
       & = \beta_0(C_0) \prod_{i=1}^{j} [\frac{\beta_i(C_i)}{\mu_{i, pr(i)}(S_i)}]
\end{align}
$$
When $j=0$, we define the product term in the last line to be 1.

Thus we can see, the clique tree measure defined by a subset of nodes is proportional to the marginal distribution over the variables appearing in these nodes. Actually, these subsets of nodes form a sub tree. This is because each $C_i$ has parent with lower numbered index, and by transitivity, they are all connected to $C_0$. So $C_0, C_1, \ldots, C_j$ form a connected subgraph within a tree, and it must be a tree as well.

Conversely, if we select a subtree with $j$ out of $n$ cliques, pick a root node within this subtree, and then we can always obtain a topological ordering of all $n$ cliques where these $j$ come first. So any subtree's clique tree measure is always proportional to the corresponding joint distribution over variables covered by them. In this aspect, **the whole tree measure, and single clique measure are just two extremes**. This is the underlying reason we can efficiently do quries outside a clique (pp. 370, 10.3.3.2 of Koller's book), or do multiple queries efficiently (pp. 371, 10.3.3.3 of Koller's book).

**THIS SUBTREE CALIBRATION PROPERTY IS SO COOL**

##### how to obtain a topological ordering of all $n$ cliques where these $j$ come first.

I can do this constructively. One algorithm is always picking one of the cliques without parent. I would argue you can always first pick all those $j$ cliques, and then work on the remaining $n-j$ ones. I can do this by induction on $j$. When $j=1$, it's trivial. When $j=k+1$, we can remove a leaf node, leaving a $k$ node subtree. By induction, we can remove all these $k$ first. At this time, we notice that this $k+1$th leaf node has no parent, and thus can be picked.

## Sum-Product
Usually, sum-product works on factor-graphs where factor-graphs are tree-structured. This is just a special case of Junction Tree algorithm. (FGs are called Bethe cluster graphs in Koller's Coursera course, pp. 22 of `Section-3-Belief-Propagation.pdf`).
![image](./SumProduct/Bethe.png)


1. Factor graphs (Bethe cluster graphs) are automatically family preserving and RIP.
2. Plus tree structure, we have a valid junction tree (clique tree).


## Max-Sum
This is just sum-product applied to another other operations, since sum-product and max-sum have common math properties (semi ring), as discussed in Jordan's book (pp. 27-28, Chapter 4).
![image](./MaxSum/max_sum_1_Jordan.png)
![image](./MaxSum/max_sum_2_Jordan.png)

Notice that, to recover a configration, a backtracking is needed, as discussed in Jordan's book. Similar issue happens in Example 13.10 (pp. 565) of Koller's book.

Using Jordan's approach, it can give you ONE configuration, although there might be more.

It should be noted that unlike computing marginal, whatever root you use in Max-Sum is irrelevant. They will all give you the same max probability. This is not inconsistent with sum-product, as actually same thing happens as well in sum-product: when you marginalize everything, it sum to 1 (or some other constant). It's just that people usually keep one variable left when doing sum-product, but marginalize everything when doing max-sum.

---

## Decomposable, recursively simplicial, triangulated, and having junction tree

There are deep connections among several concepts: decomposable, recursively simplicial, triangulated (chordal), and having junction tree.

The proofs of all these are given in Thm 1 of [GraphNote](./graphnotes.pdf). It's good to know them if you want to follow some proofs in Koller's book.

I want to make some clarification and comments for these proofs.

### Lemma 1

* Step 1: trivial
* Step 2: when we say minimal set, it doesn't mean it has smallest number of nodes to separate. It just means that given this set, if we remove any of them from the set, we can't separate. Consider a graph with 4 layers, each layer fully connected to adjacent layers. Layer 1 has 1 node, layer 2 has 2, layer 3 has 3, layer 4 has 4. Then actually to separate nodes in 1 and nodes in 4, we can choose nodes in layer 2, or nodes in layer 3. Both are minimal sets. To be more precise, consider the set of paths connecting $a$ and $b$, then a minimal set is a set of nodes that has intersection with all these paths, and removing one node won't do. This means each node in the minimal set must lie on some path connecting $a$ and $b$. Also, for each element in the minimal set, there must exist path where the only intersection with this minimal set is this element. Otherwise, we could always remove this element, still blocking all paths.
* Step 3:
    - 3a: easy
    - 3b: to make it more easy, we can make the arugment easier to follow. Instead of finding some general path, for $A$, there must be a path from $a$ to $u$, where we only use nodes in $A$, except for last node (which is $u$), and similar for $v$, and similarly for $B$. This follows from my explanation for step 2. Thus, we can have a cycle $a - u - b - v - a$ (well not necessarily $a$ and $b$, as they may get reduced if we take shortest path), and it must have chord. This is what's done in Lemma 2 of Jordan's book (chapter 17).
    - 3c: there can be no chord except within $S$. There can be chord between $A$ and $B$ without crossing $S$, since by our construction, $S$ separates $A$ and $B$.
* Step 4: Indeed, take $A\cup S$ as example. If this subgraph have chordless cycle $x_1 - x_2 - ... - x_n$, then consider the corresponding cycle in $G$. The edges among these vertices are exactly the same for $A\cup S$ and $G$. So $G$ is not chordal, violating our assumption.

In addition, it is basically Lemma 2 in Jordan's book (pp. 25, Chapter 17) and first half of Lemma 3 in Jordan's book (pp. 25, Chapter 17).

### Lemma 2

* Step 1. It says that when a graph is not complete, it always has two simplical nodes, one in $A$, one in $C$.
* Step 2. This part is very strong. It says that when removing any single node from a decomposable graph, it remains decomposable (so you can remove any number). it keeps trying aruging that when removing any edge, either from $A$, $B$, or $C$, the resultant subgraphs are decomposable. This is because it tries to match the definition of decomposable given in first page of notes. Notice that to be decomposable, $B$ should separate $A$ and $C$, and this even holds after you remove something in $B$.
  - With such strong proposition, when we remove a simplical node, definitely the graph stays decomposable, and by induction (decomposable graph with $n-1$ nodes are simplicial), it's recursively simplicial.
  
#### previous explanation. 
step 1 is second half of Lemma 3 in Jordan's book (pp. 25, Chapter 17). This establishes that decomposable graph has a simplical vertex.

I want to make clarification for Step 2, in that when they divide variables into A,B,C (where B can be empty), they want to say that no matter where we remove a point, the remaining thing still is decomposable, in terms of satisfying the definition of decomposable given earlier in that document. Since union(A,B) and union(B,C) are still decomposable, and also that B still separates A and C (even we remove a point from B, which makes the graph having fewer edges, so separation still holds; notice that B can be empty, so we may not be able to remove a point from B).


### Lemma 3

This is basically Theorem 3 in Jordan's book (pp. 25, Chapter 17). We are using the *property of simplicial, such that when adding back the node, it won't create or affect other (maximal) cliques, other than the one containing its neighbors*. Consider the max cliques after adding back the vertex. If those max cliques are only related to non-$v$ nodes, then it must be already in some node of $T'$. Otherwise, the clique in the full (original) graph must involve $v$, and the only max clique involving it is the one formed by $v$ and its neighbors. All other (probably non-max) cliques involving $v$ must be be contained in this max clique, due to $v$ being simplicial. Since nodes of any clique involving $v$ must be its neighbors, and they are all contained in that maximal clique for $v$.

Notice that the neighbors of $v$ can't be empty, since we assume that the whole graph is connected, and $v$ must have some neighbors.


### Lemma 4

* Step 1. If this is case, when we just apply inductive hypothesis.
* Step 2. Using Lemma 1 in Jordan's book, we know that the sepset between $C \cap C'$ separates $R$ from $V-C$. Consider the cliques in $G'$. all cliques in it do not involve $R$, and they must be cliques in the original $G$ as well, since $G'$ has only fewer edges than $G$. So they must be in $T'$, as $T'$ includes all cliques in $G$ having no intersection with $R$, by definition of clique tree (every clique is in the tree). So $T'$ is a junction tree for $G'$.
* Step 3. Consider how can a cycle in $G$ appear. Since we divide all variables into 3 sets R (non empty), inter(C,C') (non empty) and V-C (can be empty).

There are 4 possiblities that the nodes for that chord can happen:
* (1) In R+inter(C,C'): no. since this is C, whose subgraph is also chordal (complete).
* (2) In inter(C,C') + V-C: no. by induction hypothesis, the subgraph corresponding to these nodes is chordal.
* (3) In R and V-C: no. Because inter(C,C') separates them, so we can't have a cycle involving only R and V-C.
* (4) In all three. In this case, since inter(C,C') separates the other two, we must have at least two nodes in the cycle coming from inter(C,C') (if inter(C,C') only has one node, then there's no cycle possible). But since inter(C,C') is completely connected, this cycle must have a chord.

## Constructing a Junction Tree

Check 17.10 (pp. 27) of Jordan's book. Basically, first find all cliques in the graph (Koller's book says there are cheap ways to do this in chrodal graph), and then run max spanning tree. The proof is easy to follow. Basically, Jordan uses (17.43), that is, counts of a variable in edges and nodes, to characterize RIP, and in the proof for Thm 5, RIP holds (that is, 17.43 becomes equality) if and only if weight sum is equal to its upper bound. And acheiving upper bound is actually equivalent to having found max spanning tree, since this bound is tight, because junction tree always exists on a chordal graph.